In [ ]:
# Evaluation script for Hugging Face classifiers

import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import classification_report, confusion_matrix


MODEL_NAME = "MB55/gbert-lora-final"
#To test other models, simply replace MODEL_NAME with any of the available models below:
#MB55/rembert-qlora
#MB55/LLaMleinclassifier
#MB55/EuroLLM-Classifier-QLoRA
#MB55/bueble-classifier
#MB55/teuken7b-advance-classifier

DATA_PATH = "data/test_split.csv"
TEXT_COLUMN = "input_text"
LABEL_COLUMN = "label"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME).to(DEVICE)


df = pd.read_csv(DATA_PATH)
texts = df[TEXT_COLUMN].tolist()
true_labels = df[LABEL_COLUMN].tolist()


pred_labels = []
for text in texts:
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(DEVICE)
    with torch.no_grad():
        outputs = model(**inputs)
    pred = outputs.logits.argmax(dim=-1).item()
    pred_labels.append(pred)


report_df = pd.DataFrame(classification_report(true_labels, pred_labels, output_dict=True)).transpose()
confmat_df = pd.DataFrame(confusion_matrix(true_labels, pred_labels))

report_df.to_csv("results/eval_report.csv")
confmat_df.to_csv("results/conf_matrix.csv")


